In [1]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect('prices.db')

# See all tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
print("Tables:", tables)

cursor = conn.cursor()
cursor.execute("SELECT * FROM prices LIMIT 5")  
print(cursor.fetchall())

conn.close()

Tables:      name
0  prices
[('2010-01-04 00:00:00', 'ITC.NS', 53.541446685791016), ('2010-01-05 00:00:00', 'ITC.NS', 54.069129943847656), ('2010-01-06 00:00:00', 'ITC.NS', 54.19579315185547), ('2010-01-07 00:00:00', 'ITC.NS', 54.026920318603516), ('2010-01-08 00:00:00', 'ITC.NS', 54.14301681518555)]


In [2]:
import sqlite3
import pandas as pd

DB_PATH = "prices.db"

conn = sqlite3.connect(DB_PATH)

# See table names
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
print("Tables:", tables['name'].tolist())

# Preview first 10 rows of 'prices' table
df = pd.read_sql_query("SELECT * FROM prices LIMIT 10", conn)
print(df)

conn.close()


Tables: ['prices']
                  date ticker_symbol      close
0  2010-01-04 00:00:00        ITC.NS  53.541447
1  2010-01-05 00:00:00        ITC.NS  54.069130
2  2010-01-06 00:00:00        ITC.NS  54.195793
3  2010-01-07 00:00:00        ITC.NS  54.026920
4  2010-01-08 00:00:00        ITC.NS  54.143017
5  2010-01-11 00:00:00        ITC.NS  54.164127
6  2010-01-12 00:00:00        ITC.NS  52.781528
7  2010-01-13 00:00:00        ITC.NS  52.549335
8  2010-01-14 00:00:00        ITC.NS  52.380466
9  2010-01-15 00:00:00        ITC.NS  53.351456


In [3]:
# Required Libraries
import sqlite3
import gspread
from google.oauth2.service_account import Credentials
from datetime import datetime
import time
import sys

In [4]:
# Path to SQLite database
DB_PATH = "prices.db"

# Path to Google Sheets credentials JSON file
CREDENTIALS_FILE = "stocksqlite-320c0219c9a7.json"

# Google Sheet ID (from the URL)
SHEET_ID = "1X9_Navxt6YO3q7ODPqTL1CK-kI7ln6oyd_rEKPDB6XM"

# Worksheet name 
WORKSHEET_NAME = "Sheet1"

# Cell references
TICKER_CELL = "A1"
DATE_CELL = "A2"
OUTPUT_CELL = "A4"

# Monitoring interval 
CHECK_INTERVAL = 2

In [5]:
# ===================================================================
# DATABASE FUNCTIONS
# ===================================================================

def get_stock_price(ticker, date_str):
    try:
        # Connect to SQLite database
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        
        # Validate date format
        try:
            datetime.strptime(date_str, '%Y-%m-%d')
        except ValueError:
            conn.close()
            return "Invalid date format"
        
        # Query database - exact date match
        query = """
        SELECT close 
        FROM prices 
        WHERE ticker_symbol = ? 
        AND date(date) = ?
        """
        
        cursor.execute(query, (ticker, date_str))
        result = cursor.fetchone()
        
        if result:
            conn.close()
            return round(result[0], 2)
        
        # If no exact match, try to find nearest date
        query_nearest = """
        SELECT date, close,
               ABS(julianday(date(date)) - julianday(?)) as date_diff
        FROM prices 
        WHERE ticker_symbol = ?
        ORDER BY date_diff
        LIMIT 1
        """
        
        cursor.execute(query_nearest, (date_str, ticker))
        nearest_result = cursor.fetchone()
        
        conn.close()
        
        if nearest_result:
            nearest_date = nearest_result[0].split()[0]  # Extract date part
            nearest_price = round(nearest_result[1], 2)
            return f"{nearest_price} (nearest: {nearest_date})"
        else:
            return "No data found"
            
    except sqlite3.Error as e:
        return f"Database error: {str(e)}"
    except Exception as e:
        return f"Error: {str(e)}"


In [6]:
def format_price(price):
    if isinstance(price, (int, float)):
        return f"₹{price:,.2f}"
    elif isinstance(price, str) and "(" in price:
        # Handle "price (nearest: date)" format
        parts = price.split(" (")
        try:
            num = float(parts[0])
            return f"₹{num:,.2f} ({parts[1]}"
        except:
            return price
    return str(price)


In [7]:
# ===================================================================
# GOOGLE SHEETS FUNCTIONS
# ===================================================================

def connect_to_google_sheets():
    try:
        # Define the scope
        scopes = [
            'https://spreadsheets.google.com/feeds',
            'https://www.googleapis.com/auth/drive'
        ]
        
        # Authenticate using service account
        credentials = Credentials.from_service_account_file(
            CREDENTIALS_FILE,
            scopes=scopes
        )
        
        # Connect to Google Sheets
        client = gspread.authorize(credentials)
        
        # Open the spreadsheet by ID
        spreadsheet = client.open_by_key(SHEET_ID)
        worksheet = spreadsheet.worksheet(WORKSHEET_NAME)
        
        return worksheet
        
    except FileNotFoundError:
        print(f"\nERROR: Credentials file '{CREDENTIALS_FILE}' not found!")
        print("   Make sure the JSON file is in the same folder as this script.\n")
        sys.exit(1)
    except gspread.exceptions.SpreadsheetNotFound:
        print(f"\nERROR: Spreadsheet not found!")
        print(f"   Sheet ID: {SHEET_ID}")
        print("   Make sure you've shared the sheet with your service account email.\n")
        sys.exit(1)
    except Exception as e:
        print(f"\nERROR connecting to Google Sheets: {str(e)}\n")
        sys.exit(1)

In [8]:
def update_closing_price(worksheet, show_messages=True):
    try:
        # Read inputs from Google Sheet
        ticker = worksheet.acell(TICKER_CELL).value
        date_str = worksheet.acell(DATE_CELL).value
        
        # Check if cells are empty
        if not ticker or ticker.strip() == '':
            if show_messages:
                print("⏳ Waiting for ticker symbol in A1...")
            return False
        
        if not date_str or date_str.strip() == '':
            if show_messages:
                print("⏳ Waiting for date in A2...")
            return False
        
        # Clean inputs
        ticker = ticker.strip().upper()
        date_str = date_str.strip()
        
        if show_messages:
            print(f"\n🔍 Fetching price...")
            print(f"   Ticker: {ticker}")
            print(f"   Date: {date_str}")
        
        # Get closing price from database
        closing_price = get_stock_price(ticker, date_str)
        
        # Format the price
        formatted_price = format_price(closing_price)
        
        # Update Google Sheet cell A4
        worksheet.update(OUTPUT_CELL, [[formatted_price]])
        
        # Format the cell based on result type
        if isinstance(closing_price, (int, float)):
            # Exact match found - green color, bold
            worksheet.format(OUTPUT_CELL, {
                "textFormat": {
                    "foregroundColor": {"red": 0, "green": 0.5, "blue": 0},
                    "fontSize": 12,
                    "bold": True
                },
                "horizontalAlignment": "LEFT"
            })
            if show_messages:
                print(f"✅ Closing Price: {formatted_price}")
        else:
            # Error or nearest date - different formatting
            if "No data found" in str(closing_price) or "Error" in str(closing_price) or "Invalid" in str(closing_price):
                # Error case - red color
                worksheet.format(OUTPUT_CELL, {
                    "textFormat": {
                        "foregroundColor": {"red": 0.8, "green": 0, "blue": 0},
                        "fontSize": 11,
                        "bold": False
                    },
                    "horizontalAlignment": "LEFT"
                })
                if show_messages:
                    print(f"❌ {formatted_price}")
            else:
                # Nearest date case - blue color
                worksheet.format(OUTPUT_CELL, {
                    "textFormat": {
                        "foregroundColor": {"red": 0, "green": 0.4, "blue": 0.8},
                        "fontSize": 11,
                        "bold": False
                    },
                    "horizontalAlignment": "LEFT"
                })
                if show_messages:
                    print(f"  {formatted_price}")
        
        return True
        
    except Exception as e:
        error_msg = f"Error: {str(e)}"
        try:
            worksheet.update(OUTPUT_CELL, [[error_msg]])
        except:
            pass
        if show_messages:
            print(f"❌ {error_msg}")
        return False

In [9]:
# ===================================================================
# MAIN MONITORING FUNCTION
# ===================================================================

def monitor_and_update(worksheet):
    print("\n" + "="*70)
    print(" AUTO-UPDATE MODE ACTIVE")
    print("="*70)
    print("\n Instructions:")
    print("   1. Open your Google Sheet")
    print("   2. Enter ticker symbol in cell A1 (e.g., ITC.NS)")
    print("   3. Enter date in cell A2 (e.g., 2010-01-04)")
    print("   4. Cell A4 will automatically show the closing price!")
    print("\n   When you change A1 or A2, A4 will update automatically.")
    print(f"\n  Checking for changes every {CHECK_INTERVAL} seconds...")
    print("   Press Ctrl+C to stop\n")
    print("="*70)
    
    last_ticker = None
    last_date = None
    first_run = True
    
    try:
        while True:
            # Read current values from A1 and A2
            current_ticker = worksheet.acell(TICKER_CELL).value
            current_date = worksheet.acell(DATE_CELL).value
            
            # Clean values
            if current_ticker:
                current_ticker = current_ticker.strip().upper()
            if current_date:
                current_date = current_date.strip()
            
            # Check if values have changed
            if (current_ticker != last_ticker or current_date != last_date):
                if current_ticker and current_date:
                    timestamp = datetime.now().strftime('%H:%M:%S')
                    print(f"\n[{timestamp}] Change detected! Updating...")
                    
                    # Update the closing price
                    update_closing_price(worksheet, show_messages=True)
                    
                    # Remember current values
                    last_ticker = current_ticker
                    last_date = current_date
                    
                elif first_run:
                    # First run - check if cells are empty
                    print("⏳ Waiting for input in cells A1 and A2...")
                    first_run = False
            
            # Wait before next check
            time.sleep(CHECK_INTERVAL)
            
    except KeyboardInterrupt:
        print("\n  Auto-update stopped by user.")
        print("   You can run the script again anytime to resume monitoring.\n")


In [10]:

# ===================================================================
# UTILITY FUNCTIONS
# ===================================================================

def verify_database():

    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        
        # Get sample tickers
        cursor.execute("SELECT DISTINCT ticker_symbol FROM prices LIMIT 5")
        tickers = cursor.fetchall()
        
        # Get date range
        cursor.execute("SELECT MIN(date(date)), MAX(date(date)) FROM prices")
        date_range = cursor.fetchone()
        
        # Get total records
        cursor.execute("SELECT COUNT(*) FROM prices")
        total = cursor.fetchone()[0]
        
        conn.close()
        
        print(f"   Database connected: {DB_PATH}")
        print(f"   Total records: {total:,}")
        print(f"   Date range: {date_range[0]} to {date_range[1]}")
        print(f"   Sample tickers: {', '.join([t[0] for t in tickers])}")
        
        return True
        
    except Exception as e:
        print(f" Database error: {str(e)}")
        print(f"   Make sure '{DB_PATH}' exists in the same folder.\n")
        return False


In [11]:
def test_single_query():
    """
    Test mode - single query without monitoring
    """
    print("\n" + "="*70)
    print("TEST MODE - Single Query")
    print("="*70)
    
    ticker = input("\nEnter ticker symbol (e.g., ITC.NS): ").strip().upper()
    date_str = input("Enter date (YYYY-MM-DD, e.g., 2010-01-04): ").strip()
    
    if not ticker or not date_str:
        print("❌ Both ticker and date are required!")
        return
    
    print("\n Querying database...")
    closing_price = get_stock_price(ticker, date_str)
    formatted_price = format_price(closing_price)
    
    print(f"\n Result:")
    print(f"   Ticker: {ticker}")
    print(f"   Date: {date_str}")
    print(f"   Closing Price: {formatted_price}")
    print()


In [12]:
# ===================================================================
# MAIN EXECUTION
# ===================================================================

def main():
    print("\n" + "="*70)
    print(" ITUS CAPITAL - STOCK PRICE RETRIEVER")
    print("="*70)
    
    # Step 1: Verify database
    print("\n[Step 1/2] Checking database...")
    if not verify_database():
        sys.exit(1)
    
    # Step 2: Connect to Google Sheets
    print("\n[Step 2/2] Connecting to Google Sheets...")
    worksheet = connect_to_google_sheets()
    print("✅ Connected to Google Sheets successfully!")
    
    # Show menu
    print("\n" + "="*70)
    print("SELECT MODE:")
    print("="*70)
    print("1. Auto-Update Mode (Recommended)")
    print("   → Automatically updates A4 when you change A1 or A2")
    print()
    print("2. Test Mode")
    print("   → Test a single query from terminal")
    print()
    print("3. Exit")
    print("="*70)
    
    choice = input("\nEnter your choice (1-3): ").strip()
    
    if choice == '1':
        monitor_and_update(worksheet)
    elif choice == '2':
        test_single_query()
    elif choice == '3':
        print("\nGoodbye!\n")
        sys.exit(0)
    else:
        print("\nInvalid choice. Please run the script again.\n")


In [ ]:
if __name__ == "__main__":
    main()


 ITUS CAPITAL - STOCK PRICE RETRIEVER

[Step 1/2] Checking database...
   Database connected: prices.db
   Total records: 15,564
   Date range: 2010-01-04 to 2025-10-03
   Sample tickers: ITC.NS, RELIANCE.NS, TCS.NS, ^NSEI

[Step 2/2] Connecting to Google Sheets...
✅ Connected to Google Sheets successfully!

SELECT MODE:
1. Auto-Update Mode (Recommended)
   → Automatically updates A4 when you change A1 or A2

2. Test Mode
   → Test a single query from terminal

3. Exit



Enter your choice (1-3):  1



 AUTO-UPDATE MODE ACTIVE

 Instructions:
   1. Open your Google Sheet
   2. Enter ticker symbol in cell A1 (e.g., ITC.NS)
   3. Enter date in cell A2 (e.g., 2010-01-04)
   4. Cell A4 will automatically show the closing price!

   When you change A1 or A2, A4 will update automatically.

  Checking for changes every 2 seconds...
   Press Ctrl+C to stop


[14:40:33] Change detected! Updating...

🔍 Fetching price...
   Ticker: RELIANCE.NS
   Date: 2010-01-09


/var/folders/4y/p5wbs2392ng1qdh47g8kk6x00000gn/T/ipykernel_7421/872826392.py:44: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(OUTPUT_CELL, [[formatted_price]])


  ₹224.21 (nearest: 2010-01-08)
